In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import DirectoryIterator, ImageDataGenerator

### Model training

In [2]:
training_set = DirectoryIterator(directory = 'Large-image-dataset/training',
                                 image_data_generator = ImageDataGenerator(rescale=1.0/255),
                                 target_size = (64, 64),
                                 batch_size = 32,
                                 shuffle = False,  
                                 class_mode = 'binary')

Found 2225 images belonging to 2 classes.


In [3]:
print(training_set.classes)
unique, counts = np.unique(training_set.classes, return_counts=True)
print(dict(zip(training_set.class_indices, counts)))

[0 0 0 ... 1 1 1]
{'Legal': 762, 'No ball': 1463}


In [4]:
print(training_set.samples)

2225


In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
import numpy as np

# Initialize empty lists for training data
X_train = []
y_train = []

# Iterate through the training set and collect the data
i=1
for x, y in training_set:
    print(i)
    i+=1
    X_train.extend(x)
    y_train.extend(y)
    if len(X_train) >= training_set.samples:
        break
# Convert the lists into numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape the image data to a 2D array (flatten each image)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE to the reshaped training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_reshaped, y_train)

# Reshape the resampled data back to its original 4D shape (64x64 images with 3 color channels)
X_resampled = X_resampled.reshape(-1, 64, 64, 3)

# Shuffle the resampled data
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

# Now you have X_resampled and y_resampled with balanced classes


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


In [6]:
print(X_resampled.shape)
print(y_resampled.shape)

(2926, 64, 64, 3)
(2926,)


In [7]:
new_datagen = ImageDataGenerator(rescale=1./255)

# Create the new training set using the resampled data
training_set_resampled = new_datagen.flow(X_resampled, y_resampled, batch_size=32)
# Convert X_resampled and y_resampled into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))

# Shuffle, batch, and optionally prefetch the data for efficient training
training_set_smote = train_dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Use this dataset in your model training

In [8]:
print(y_resampled)
unique, counts = np.unique(y_resampled, return_counts=True)
print(dict(zip(unique, counts)))

[1. 1. 1. ... 1. 1. 1.]
{0.0: 1463, 1.0: 1463}


In [9]:
validation_set = DirectoryIterator(directory = 'Large-image-dataset/validation',
                                   image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                   target_size = (64, 64),
                                   batch_size = 32,
                                   shuffle = False,
                                   class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [10]:
cnn = tf.keras.models.Sequential()

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [13]:
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [14]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [15]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
cnn.fit(x = training_set_smote, validation_data = validation_set, epochs = 10)

Epoch 1/10
92/92 [==============================] - 9s 89ms/step - loss: 0.5859 - accuracy: 0.6774 - val_loss: 0.4751 - val_accuracy: 0.7710
Epoch 2/10
92/92 [==============================] - 6s 68ms/step - loss: 0.3577 - accuracy: 0.8401 - val_loss: 0.3147 - val_accuracy: 0.9122
Epoch 3/10
92/92 [==============================] - 7s 75ms/step - loss: 0.2139 - accuracy: 0.9170 - val_loss: 0.3498 - val_accuracy: 0.9046
Epoch 4/10
92/92 [==============================] - 8s 84ms/step - loss: 0.1179 - accuracy: 0.9552 - val_loss: 0.2630 - val_accuracy: 0.9466
Epoch 5/10
92/92 [==============================] - 7s 73ms/step - loss: 0.0854 - accuracy: 0.9686 - val_loss: 0.2815 - val_accuracy: 0.9504
Epoch 6/10
92/92 [==============================] - 7s 79ms/step - loss: 0.0505 - accuracy: 0.9853 - val_loss: 0.3961 - val_accuracy: 0.9427
Epoch 7/10
92/92 [==============================] - 8s 86ms/step - loss: 0.0477 - accuracy: 0.9853 - val_loss: 0.3769 - val_accuracy: 0.9733
Epoch 8/10
92

In [17]:
cnn.save('final_model.keras')

### Model testing

In [18]:
from sklearn.metrics import classification_report

loaded_model = tf.keras.models.load_model(r'final_model.keras')

In [19]:
loss, accuracy = loaded_model.evaluate(training_set)

print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

y_pred = loaded_model.predict(training_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(training_set.classes.tolist(), y_pred))

70/70 [==============================] - 18s 251ms/step - loss: 0.0031 - accuracy: 1.0000
Training Loss: 0.0030798546504229307
Training Accuracy: 1.0
70/70 [==============================] - 16s 228ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       762
           1       1.00      1.00      1.00      1463

    accuracy                           1.00      2225
   macro avg       1.00      1.00      1.00      2225
weighted avg       1.00      1.00      1.00      2225



In [20]:
loss, accuracy = loaded_model.evaluate(validation_set)

print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

y_pred = loaded_model.predict(validation_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(validation_set.classes.tolist(), y_pred))

9/9 [==============================] - 2s 226ms/step - loss: 0.4141 - accuracy: 0.9771
Validation Loss: 0.4141470193862915
Validation Accuracy: 0.9770992398262024
9/9 [==============================] - 2s 233ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       129
           1       0.96      0.99      0.98       133

    accuracy                           0.98       262
   macro avg       0.98      0.98      0.98       262
weighted avg       0.98      0.98      0.98       262



In [21]:
testing_set = DirectoryIterator(directory = 'Large-image-dataset/testing',
                                image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                target_size = (64, 64),
                                batch_size = 32,
                                shuffle = False,
                                class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [22]:
loss, accuracy = loaded_model.evaluate(testing_set)

print("Testing Loss:", loss)
print("Testing Accuracy:", accuracy)

y_pred = loaded_model.predict(testing_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(testing_set.classes.tolist(), y_pred))

9/9 [==============================] - 2s 231ms/step - loss: 0.4141 - accuracy: 0.9771
Testing Loss: 0.4141470193862915
Testing Accuracy: 0.9770992398262024
9/9 [==============================] - 2s 226ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       129
           1       0.96      0.99      0.98       133

    accuracy                           0.98       262
   macro avg       0.98      0.98      0.98       262
weighted avg       0.98      0.98      0.98       262



In [23]:
print(testing_set.classes.tolist())
print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [25]:
# Load the test image
image_location = r'test\legal.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 185ms/step
[[0.]]
Legal Ball


In [26]:
# Load the test image
image_location = r'test\noball_1.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 30ms/step
[[1.]]
No Ball


In [27]:
# Load the test image
image_location = r'test\noball_2.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 29ms/step
[[1.]]
No Ball


In [28]:
testing_set = DirectoryIterator(directory = 'Small-image-dataset',
                                image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                target_size = (64, 64),
                                batch_size = 32,
                                shuffle = False,
                                class_mode = 'binary')

Found 77 images belonging to 2 classes.


In [29]:
loss, accuracy = loaded_model.evaluate(testing_set)

print("Testing Loss:", loss)
print("Testing Accuracy:", accuracy)

y_pred = loaded_model.predict(testing_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(testing_set.classes.tolist(), y_pred))

3/3 [==============================] - 1s 94ms/step - loss: 1.7590 - accuracy: 0.7143
Testing Loss: 1.7590413093566895
Testing Accuracy: 0.7142857313156128
3/3 [==============================] - 1s 81ms/step
              precision    recall  f1-score   support

           0       0.56      0.56      0.56        25
           1       0.79      0.79      0.79        52

    accuracy                           0.71        77
   macro avg       0.67      0.67      0.67        77
weighted avg       0.71      0.71      0.71        77



In [30]:
testing_set = DirectoryIterator(directory = 'Final-test',
                                image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                target_size = (64, 64),
                                batch_size = 32,
                                shuffle = False,
                                class_mode = 'binary')

Found 10 images belonging to 2 classes.


In [31]:
loss, accuracy = loaded_model.evaluate(testing_set)

print("Testing Loss:", loss)
print("Testing Accuracy:", accuracy)

y_pred = loaded_model.predict(testing_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(testing_set.classes.tolist(), y_pred))

1/1 [==============================] - 0s 114ms/step - loss: 0.0105 - accuracy: 1.0000
Testing Loss: 0.010458553209900856
Testing Accuracy: 1.0
1/1 [==============================] - 0s 109ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [32]:
print(tf.version.VERSION)

2.10.0
